In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp

import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from annotation_utilities import *
from registration_utilities import *

from skimage.measure import mesh_surface_area, marching_cubes, correct_mesh_orientation

from itertools import izip
import pandas as pd

import time

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
volume_dir = '/home/yuncong/CSHL_volumes/'
mesh_rootdir = create_if_not_exists('/home/yuncong/CSHL_meshes')

In [3]:
with open('colors.txt', 'r') as f:
    colors = {l[0]: np.r_[float(l[1]), float(l[2]), float(l[3])] for l in map(lambda x: x.split(), f.readlines())}

In [4]:
volume_landmark_names_unsided = ['12N', '5N', '6N', '7N', '7n', 'AP', 'Amb', 'LC',
                                 'LRt', 'Pn', 'R', 'RtTg', 'Tz', 'VLL', 'sp5']
linear_landmark_names_unsided = ['outerContour']

labels_unsided = volume_landmark_names_unsided + linear_landmark_names_unsided
labels_unsided_indices = dict((j, i+1) for i, j in enumerate(labels_unsided))  # BackG always 0

labelMap_unsidedToSided = {'12N': ['12N'],
                            '5N': ['5N_L', '5N_R'],
                            '6N': ['6N_L', '6N_R'],
                            '7N': ['7N_L', '7N_R'],
                            '7n': ['7n_L', '7n_R'],
                            'AP': ['AP'],
                            'Amb': ['Amb_L', 'Amb_R'],
                            'LC': ['LC_L', 'LC_R'],
                            'LRt': ['LRt_L', 'LRt_R'],
                            'Pn': ['Pn_L', 'Pn_R'],
                            'R': ['R_L', 'R_R'],
                            'RtTg': ['RtTg'],
                            'Tz': ['Tz_L', 'Tz_R'],
                            'VLL': ['VLL_L', 'VLL_R'],
                            'sp5': ['sp5'],
                           'outerContour': ['outerContour']}

labelMap_sidedToUnsided = {n: nu for nu, ns in labelMap_unsidedToSided.iteritems() for n in ns}

from itertools import chain
labels_sided = list(chain(*(labelMap_unsidedToSided[name_u] for name_u in labels_unsided)))
labels_sided_indices = dict((j, i+1) for i, j in enumerate(labels_sided)) # BackG always 0

In [5]:
atlas_volume = bp.unpack_ndarray_file(volume_dir + '/atlasVolume_icp.bp').astype(np.int8)

available_labels_sided = [labels_sided[i-1] for i in np.unique(atlas_volume) if i > 0]
available_labels_unsided = set([labelMap_sidedToUnsided[name] for name in available_labels_sided ])

In [ ]:
# Download volumes

for i, stack in enumerate(['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']):
    volume_d = volume_dir + '/%(stack)s' % {'stack': stack}
    create_if_not_exists(volume_d)
    os.system('scp gcn:/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/%(stack)s/%(stack)s_atlasProjectedVolume.bp %(volume_d)s/' % {'stack': stack, 'volume_d':volume_d})
    os.system('scp gcn:/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/%(stack)s/%(stack)s_localAdjustedVolume.bp %(volume_d)s/' % {'stack': stack, 'volume_d':volume_d})
    
#     for name_s in available_labels_unsided:
#         os.system('scp gcn:/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/%(stack)s/%(stack)s_scoreVolume_%(name_s)s.bp %(volume_d)s/' % \
#                   {'stack': stack, 'volume_d':volume_d, 'name_s': name_s})

In [6]:
def fill_sparse_volume(volume_sparse):

    volume = np.zeros_like(volume_sparse, np.int8)

    for z in range(volume_sparse.shape[-1]):
        for ind, cnts in find_contour_points(volume_sparse[..., z]).iteritems():
            cnt = cnts[np.argsort(map(len, cnts))[-1]]
            pts = points_inside_contour(cnt)
            volume[pts[:,1], pts[:,0], z] = ind
    return volume

In [ ]:
for stack in ['MD589', 'MD594', 'MD585', 'MD593', 'MD592', 'MD590', 'MD591', 'MD595', 'MD598', 'MD602', 'MD603']:
    mesh_dir = create_if_not_exists(os.path.join(mesh_rootdir, stack))
    
    #################
    
    volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_atlasProjectedVolume.bp' % {'stack':stack}).astype(np.int8)
    atlasProjected_volume = fill_sparse_volume(volume_sparse)

    atlasProjectedVolume_landmark_bboxes = {name: bbox_3d(atlasProjected_volume == labels_sided_indices[name])
                               for name in available_labels_sided}
    
    ###################
    
    for name_s in available_labels_sided:

        print name_s

        xmin, xmax, ymin, ymax, zmin, zmax = atlasProjectedVolume_landmark_bboxes[name_s]

        vol = (atlasProjected_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)

        t = time.time()

        polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

        sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5

        save_mesh_stl(polydata, mesh_dir + "/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})
        
    ####################
        
    volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_localAdjustedVolume.bp' % {'stack':stack}).astype(np.int8)
    localAdjusted_volume = fill_sparse_volume(volume_sparse)

    localAdjustedVolume_landmark_bboxes = {name: bbox_3d(localAdjusted_volume == labels_sided_indices[name])
                               for name in available_labels_sided}

    ######################
    
    for name_s in available_labels_sided:

        print name_s

        xmin, xmax, ymin, ymax, zmin, zmax = localAdjustedVolume_landmark_bboxes[name_s]

        vol = (localAdjusted_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)

        t = time.time()

        polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)

        sys.stderr.write('time: %.2f\n' % (time.time() - t) )

        save_mesh_stl(polydata, mesh_dir + "/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})

In [54]:
stack = 'MD591'
mesh_dir = create_if_not_exists(os.path.join(mesh_rootdir, stack))

In [55]:
volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_atlasProjectedVolume.bp' % {'stack':stack}).astype(np.int8)
atlasProjected_volume = fill_sparse_volume(volume_sparse)
del volume_sparse

atlasProjectedVolume_landmark_bboxes = {name: bbox_3d(atlasProjected_volume == labels_sided_indices[name])
                           for name in available_labels_sided}

In [ ]:
for name_s in available_labels_sided:

    print name_s

    xmin, xmax, ymin, ymax, zmin, zmax = atlasProjectedVolume_landmark_bboxes[name_s]

    vol = (atlasProjected_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)
    
    t = time.time()
    polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)
    sys.stderr.write('time: %.2f\n' % (time.time() - t) ) # ~ 150s for outerContour; 93s for sp5
    
    save_mesh_stl(polydata, mesh_dir + "/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})

In [56]:
volume_sparse = bp.unpack_ndarray_file(volume_dir + '%(stack)s/%(stack)s_localAdjustedVolume.bp' % {'stack':stack}).astype(np.int8)
localAdjusted_volume = fill_sparse_volume(volume_sparse)
del volume_sparse

localAdjustedVolume_landmark_bboxes = {name: bbox_3d(localAdjusted_volume == labels_sided_indices[name])
                           for name in available_labels_sided}

In [57]:
for name_s in available_labels_sided:

    print name_s

    xmin, xmax, ymin, ymax, zmin, zmax = localAdjustedVolume_landmark_bboxes[name_s]

    vol = (localAdjusted_volume[ymin:ymax+1, xmin:xmax+1, zmin:zmax+1] == labels_sided_indices[name_s]).astype(np.float)
    
    t = time.time()
    polydata = volume_to_polydata(vol, origin=(xmin,ymin,zmin), num_simplify_iter=3, smooth=True)
    sys.stderr.write('time: %.2f\n' % (time.time() - t) )
    
    save_mesh_stl(polydata, mesh_dir + "/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s})

12N
area: 49886.66

marching cube: 0.18 seconds
compute surface area: 0.01 seconds
fill point array: 0.07 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.07 seconds
simplify 0 @ 59933: 0.50 seconds
simplify 1 @ 11988: 0.58 seconds
simplify 2 @ 2399: 0.11 seconds



5N_L

time: 1.47
marching cube: 0.09 seconds



area: 29714.46

...


VLL_R

time: 1.52
marching cube: 0.15 seconds



area: 43291.55

compute surface area: 0.01 seconds
fill point array: 0.06 seconds
fill cell array: 0.00 seconds
mesh_to_polydata: 0.06 seconds
simplify 0 @ 38026: 0.52 seconds
simplify 1 @ 7606: 0.37 seconds
simplify 2 @ 1522: 0.07 seconds


time: 1.19


In [ ]:
# Load atlasProjected volume

atlasProjected_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_atlasProjectedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    atlasProjected_polydata_list[name_s] = polydata

In [58]:
# Load localAdjusted volume

localAdjusted_polydata_list = {}

for name_s in available_labels_sided:

    fn = mesh_rootdir + "/%(stack)s/%(stack)s_localAdjustedVolume_%(name)s_smoothed.stl" % {'stack': stack, 'name': name_s}

    reader = vtk.vtkSTLReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()    
    localAdjusted_polydata_list[name_s] = polydata

In [59]:
ren1 = vtk.vtkRenderer()

renWin1 = vtk.vtkRenderWindow()
renWin1.AddRenderer(ren1)

iren1 = vtk.vtkRenderWindowInteractor()
iren1.SetRenderWindow(renWin1)

camera = vtk.vtkCamera()

# 45 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-20, -30, -10)
camera.SetFocalPoint(1, 1, 1)

# # saggital
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(0, 0, -2)
# camera.SetFocalPoint(0, 0, 1)

# # coronal
# camera.SetViewUp(0, -1, 0)
# camera.SetPosition(-2, 0, 0)
# camera.SetFocalPoint(-1, 0, 0)

# # horizontal
# camera.SetViewUp(0, 0, -1)
# camera.SetPosition(0, 1, 0)
# camera.SetFocalPoint(0, -1, 0)

# for i, (name_s, polydata) in enumerate(atlasProjected_polydata_list.iteritems()):
    
#     m = vtk.vtkPolyDataMapper()
#     m.SetInputData(polydata)

#     a = vtk.vtkActor()
#     a.SetMapper(m)
#     a.GetProperty().SetRepresentationToWireframe()
# #     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
#     a.GetProperty().SetColor((1.,1.,1.))
    
#     a.GetProperty().SetOpacity(.1)
    
#     if name_s == 'outerContour':
#         a.GetProperty().SetOpacity(.1)
            
#     ren1.AddActor(a)

    
for i, (name_s, polydata) in enumerate(localAdjusted_polydata_list.iteritems()):
    
    m = vtk.vtkPolyDataMapper()
    m.SetInputData(polydata)

    a = vtk.vtkActor()
    a.SetMapper(m)
    a.GetProperty().SetRepresentationToWireframe()
#     a.GetProperty().SetColor(colors[labelMap_sidedToUnsided[name_s]])
    a.GetProperty().SetColor((1.,0.,0.))

    if name_s == 'outerContour':
        a.GetProperty().SetOpacity(.1)
            
    ren1.AddActor(a)
    

############ Show score volume ###########    

# for name in available_labels_unsided:
# for name in ['7N']:

#     score_volume = bp.unpack_ndarray_file(volume_dir + '/%(stack)s/%(stack)s_scoreVolume_%(name)s.bp' % \
#                                           {'stack':stack, 'name':name})
#     score_imagedata = volume_to_imagedata(score_volume)

#     volumeMapper = vtk.vtkSmartVolumeMapper()
#     #     volumeMapper.SetBlendModeToComposite()
#     volumeMapper.SetInputData(score_imagedata)

#     volumeProperty = vtk.vtkVolumeProperty()
#     #     volumeProperty.ShadeOff()
#     # volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

#     compositeOpacity = vtk.vtkPiecewiseFunction()
#     compositeOpacity.AddPoint(0.0, 0.0)
#     compositeOpacity.AddPoint(0.95, 0.01)
#     compositeOpacity.AddPoint(1.0, 1.0)
#     volumeProperty.SetScalarOpacity(compositeOpacity)

#     color = vtk.vtkColorTransferFunction()
# #     c = colors[name]
#     c = (1., 1., 1.)
#     color.AddRGBPoint(0.0, c[0], c[1], c[2])
#     color.AddRGBPoint(255.0, c[0], c[1], c[2])
#     volumeProperty.SetColor(color)

#     a = vtk.vtkVolume()
#     a.SetMapper(volumeMapper)
#     a.SetProperty(volumeProperty)

#     ren1.AddActor(a)
    
    
ren1.SetActiveCamera(camera)

widget1 = add_axes(iren1)
renWin1.Render()

ren1.ResetCamera()
renWin1.SetWindowName('original')

iren1.Start()